# AHRSFactor

## Overview

The `AHRSFactor` class implements a factor to implement an *Attitude and Heading Reference System* (AHRS) within GTSAM. It is a binary factor, taking preintegrated measurements from a gyroscope between two discrete time steps, typically denoted as $t_i$ and $t_j$. These preintegrated measurements encapsulate the rotational motion observed by an inertial measurement unit (IMU) between these two timestamps.

The `AHRSFactor` thus constrains two attitude states (represented as elements of $SO(3)$) based solely on gyroscope measurements. Accelerometer or magnetometer aiding, needed to build a complete AHRS system, must be handled separately.

## Mathematical Formulation

The `AHRSFactor` relies on the use of `PreintegratedRotation`, which applies successive exponential maps to each individual gyroscope measurement $\omega$ over the interval $[t_i, t_j]$. In this approach, every measurement contributes a small rotation given by $\exp(\omega_k \Delta t)$, and the overall preintegrated rotation is obtained by composing these incremental rotations:
$$
\Delta R_{ij}^{meas} = \prod_{k} \exp(\omega_k \Delta t)
$$

Given two estimated rotations $R_i$ at time $t_i$ and $R_j$ at time $t_j$, the factor enforces:
$$
R_j \approx R_i \cdot \Delta R_{ij}^{meas}
$$

The error term optimized by the factor graph is the rotational discrepancy captured by the logarithmic map:
$$
e = \text{Log}\left((\Delta R_{ij}^{meas})^T \cdot R_i^T R_j\right)
$$

Note: the reality is a bit more complicated, because the code also takes into account the effects of bias, and if desired, coriolis forces.

## Key Functionality

### Constructor

The constructor initializes the factor using a preintegrated AHRS measurement object, relating orientation states at discrete time instances $t_i$ and $t_j$.

### Core Methods

- `evaluateError`: calculates the error between two estimated orientations at times $t_i$ and $t_j$, relative to the preintegrated gyroscopic measurements. The error is computed as:

  $$
  \text{error} = \text{Log}\left((\Delta R_{ij}^{meas})^T \cdot R_i^T R_j\right)
  $$

  Here:

  - $R_i, R_j$ are the estimated rotation matrices at times $t_i$ and $t_j$.
  - $\Delta R_{ij}^{meas}$ is the rotation matrix obtained by integrating gyroscope measurements from $t_i$ to $t_j$.
  - $\text{Log}(\cdot)$ is the logarithmic map from $SO(3)$ to $\mathbb{R}^3$.

  The resulting 3-dimensional error vector represents the rotational discrepancy.

- `print`: outputs a readable representation of the internal state of the factor, including the associated time steps and preintegrated measurements, aiding debugging and verification.

- `equals` determines if another `AHRSFactor` instance is identical, useful in testing scenarios or when verifying the correctness of factor graph constructions.

## Typical Use Case

`AHRSFactor` is typically applied within factor-graph-based orientation estimators where orientation states are discretized in time. Its primary role is to incorporate gyroscope information in a computationally efficient and numerically stable way, especially useful in IMU-centric navigation scenarios like:

- Robotics and autonomous vehicles
- UAV navigation systems
- Motion capture and tracking systems